**1.Download S&P 500 Data**

In [46]:
import pandas as pd
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')



In [32]:
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date= '2024-09-30'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)
print(start_date)

2014-10-03 00:00:00


In [38]:
df= yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()


[*********************100%***********************]  503 of 503 completed


working with all these huge rows and colums data would be inefficient so we would use df.stack

In [43]:

df.index.names=['date','ticker']
df.columns=df.columns.str.lower()
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467003   40.708157   40.779686   
                          AAPL     22.113323   24.905001   25.052500   
                          ABBV     38.515575   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0  
                          AAPL     24.760000   24.860001  173878400.0  
                          ABBV     57.340000   57.419998   11675600.0  
                          ABT      41.279999   41.410000    3430900.0  
                          ACGL     18.320000   18.333332    1470300.0  
...                                      ...         ...          ...  
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0  
                          YUM     136.990005  137.520004    2778300.0  
                          ZBH     108.190002  108.769997    1166200.0  
                          ZBRA    368.269989  372.299988     213600.0  
                          ZTS     194.419998  195.649994    1343700.0  

[1225998 rows x 6 columns]

<!-- # ### # ###### 2.CALCULATE FEATURES AND TECHNICAL INDICATORS FOR EACH STOCK
1**) garman-klass volatility
2)RSI
3)Bollinger bands
4)ATR
5)MACD
6)Dollar volume**
GK = sqrt((0.5 * (log(high / low))^2) - ((2 * log(2) - 1) * (log(close / open))^2))(garman klass simplified in code)

 -->


In [44]:
# **** GARMAN-KLASS VOLATILITY****

df['garman_klass_vol']=((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-(np.log(df['open'])))**2
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467003   40.708157   40.779686   
                          AAPL     22.113323   24.905001   25.052500   
                          ABBV     38.515575   58.689999   58.779999   
                          ABT      34.355881   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol  
date                      ticker                    
2014-10-03 00:00:00+00:00 A              -0.001832  
                          AAPL           -0.005226  
                          ABBV           -0.061293  
                          ABT            -0.013375  
                          ACGL            0.000021  
...                                            ...  
2024-09-27 00:00:00+00:00 XYL             0.000172  
                          YUM             0.000124  
                          ZBH             0.000147  
                          ZBRA            0.000082  
                          ZTS             0.000117  

[1225998 rows x 7 columns]

In [45]:
df['rsi']=df.groupby(level=1)['adj close'].transform(lambda x : ta.rsi(close=x , length=20))

NameError: name 'pandas_ta' is not defined